In [2]:
import numpy as np
import pandas as pd

In [3]:
hydrate=pd.read_csv('input/s2-hydrate.csv',index_col=0)
cages=pd.read_csv('input/s2-cages.csv')
thf=pd.read_csv('input/thf.csv')

In [4]:
hydrate_ox=pd.DataFrame()
hydrate_ox[['X','Y','Z']]=hydrate[['OX','OY','OZ']]
hydrate_ox.insert(0,'ELEMENT','OW')
hydrate_hy=pd.DataFrame()
hydrate_hy[['X','Y','Z']]=hydrate[['H1X','H1Y','H1Z']]
hydrate_hy.insert(0, 'ELEMENT', 'HW1')
hydrate_hz=pd.DataFrame()
hydrate_hz[['X','Y','Z']]=hydrate[['H2X','H2Y','H2Z']]
hydrate_hz.insert(0, 'ELEMENT', 'HW2')

In [14]:
hydrate_total = pd.DataFrame(columns=['ELEMENT', 'X', 'Y', 'Z', 'RESIDUE_NUM'])
residue_num = 1
for row in range(len(hydrate_ox.index)):
    water = pd.concat([hydrate_ox.iloc[row], hydrate_hy.iloc[row], hydrate_hz.iloc[row]], axis=1, ignore_index=True).T
    water['RESIDUE_NUM'] = residue_num
    hydrate_total = pd.concat([hydrate_total, water], axis=0, ignore_index=True)
    residue_num += 1
hydrate_total['MOL'] = 'HOH'
hydrate_total

,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH
3,OW,5.32179,5.32179,8.5662,2,HOH
4,HW1,5.53914,5.52632,7.65672,2,HOH
...,...,...,...,...,...,...
403,HW1,4.86407,13.52592,13.58037,135,HOH
404,HW2,3.79358,12.45544,13.57796,135,HOH
405,OW,12.98252,4.31665,12.9933,136,HOH
406,HW1,12.44593,3.72957,13.52588,136,HOH


In [12]:
tip4p_hoh = hydrate_total.loc[hydrate_total['RESIDUE_NUM'] == 1]
tip4p_hoh

,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH


In [17]:
def virtual_coord(x1, x2, x3):
    """
    Returns the virtual coordinate (or 4th point) for a TIP4P water using the following alogrithm:
    x4 = x1 + a*(x2-x1) + b*(x3-x1)
    where a,b = 0.128012065

    Refer to OPLSAA TIP4P itp file in GROMACS for more information

    Author: Gabe Miles
    """
    a = b = 0.128012065
    return x1 + a * (x2 -x1) + b * (x3 - x1)

In [13]:
tip4p_hoh.loc[len(tip4p_hoh.index)] = ['MW', 0, 0, 0, 1, 'HOH']
tip4p_hoh

/tmp/ipykernel_16473/575745695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tip4p_hoh.loc[len(tip4p_hoh.index)] = ['MW', 0, 0, 0, 1, 'HOH']


,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH
3,MW,0,0,0,1,HOH


In [26]:
tip4p_hoh['X'].loc[tip4p_hoh['ELEMENT'] == 'MW'] = virtual_coord(tip4p_hoh['X'].loc[tip4p_hoh['ELEMENT'] == 'OW'][0], 
                                                                 tip4p_hoh['X'].loc[tip4p_hoh['ELEMENT'] == 'HW1'][1],
                                                                 tip4p_hoh['X'].loc[tip4p_hoh['ELEMENT'] == 'HW2'][2])

tip4p_hoh['Y'].loc[tip4p_hoh['ELEMENT'] == 'MW'] = virtual_coord(tip4p_hoh['Y'].loc[tip4p_hoh['ELEMENT'] == 'OW'][0], 
                                                                 tip4p_hoh['Y'].loc[tip4p_hoh['ELEMENT'] == 'HW1'][1],
                                                                 tip4p_hoh['Y'].loc[tip4p_hoh['ELEMENT'] == 'HW2'][2])

tip4p_hoh['Z'].loc[tip4p_hoh['ELEMENT'] == 'MW'] = virtual_coord(tip4p_hoh['Z'].loc[tip4p_hoh['ELEMENT'] == 'OW'][0], 
                                                                 tip4p_hoh['Z'].loc[tip4p_hoh['ELEMENT'] == 'HW1'][1],
                                                                 tip4p_hoh['Z'].loc[tip4p_hoh['ELEMENT'] == 'HW2'][2])
tip4p_hoh

/tmp/ipykernel_16473/462234224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tip4p_hoh['X'].loc[tip4p_hoh['ELEMENT'] == 'MW'] = virtual_coord(tip4p_hoh['X'].loc[tip4p_hoh['ELEMENT'] == 'OW'][0],
/tmp/ipykernel_16473/462234224.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tip4p_hoh['Y'].loc[tip4p_hoh['ELEMENT'] == 'MW'] = virtual_coord(tip4p_hoh['Y'].loc[tip4p_hoh['ELEMENT'] == 'OW'][0],
/tmp/ipykernel_16473/462234224.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH
3,MW,6.021258,6.023186,5.865343,1,HOH


In [27]:
%matplotlib qt5

In [29]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=tip4p_hoh['X'].loc[tip4p_hoh['ELEMENT'] != 'MW'].values,
           ys=tip4p_hoh['Y'].loc[tip4p_hoh['ELEMENT'] != 'MW'].values,
           zs=tip4p_hoh['Z'].loc[tip4p_hoh['ELEMENT'] != 'MW'].values,
           c='red')
ax.scatter(xs=tip4p_hoh['X'].loc[tip4p_hoh['ELEMENT'] == 'MW'].values,
           ys=tip4p_hoh['Y'].loc[tip4p_hoh['ELEMENT'] == 'MW'].values,
           zs=tip4p_hoh['Z'].loc[tip4p_hoh['ELEMENT'] == 'MW'].values,
           c='blue')


Figure shows that the 4th point in the model is in the correct location. Ready to move forward and add 4th point to total model.

In [43]:
tip4p_hydrate = pd.DataFrame(columns=['ELEMENT', 'X', 'Y', 'Z', 'RESIDUE_NUM'])
mw_x_coord = virtual_coord(hydrate_ox['X'], hydrate_hy['X'], hydrate_hz['X'])
mw_y_coord = virtual_coord(hydrate_ox['Y'], hydrate_hy['Y'], hydrate_hz['Y'])
mw_z_coord = virtual_coord(hydrate_ox['Z'], hydrate_hy['Z'], hydrate_hz['Z'])

hydrate_mw = pd.DataFrame({'X': mw_x_coord,
                           'Y': mw_y_coord,
                           'Z': mw_z_coord})
hydrate_mw.insert(0,'ELEMENT','MW')
hydrate_mw

,ELEMENT,X,Y,Z
1,MW,6.021258,6.023186,5.865343
2,MW,5.452710,5.300656,8.496106
3,MW,7.681844,4.168606,9.780210
4,MW,9.508757,4.288440,7.644088
5,MW,8.616587,5.338877,5.181544
...,...,...,...,...
132,MW,0.000000,8.804999,8.655000
133,MW,8.655000,0.000000,8.505001
134,MW,13.132513,12.992332,4.317725
135,MW,4.327824,12.992666,13.143288


In [44]:
tip4p_hydrate_total = pd.DataFrame(columns=['ELEMENT', 'X', 'Y', 'Z', 'RESIDUE_NUM'])
residue_num = 1
for row in range(len(hydrate_ox.index)):
    water = pd.concat([hydrate_ox.iloc[row], hydrate_hy.iloc[row], hydrate_hz.iloc[row], hydrate_mw.iloc[row]],
                       axis=1, ignore_index=True).T
    water['RESIDUE_NUM'] = residue_num
    tip4p_hydrate_total = pd.concat([tip4p_hydrate_total, water], axis=0, ignore_index=True)
    residue_num += 1
tip4p_hydrate_total['MOL'] = 'HOH'
tip4p_hydrate_total

,ELEMENT,X,Y,Z,RESIDUE_NUM,MOL
0,OW,5.92487,5.92487,5.92487,1,HOH
1,HW1,5.75998,6.83792,5.68955,1,HOH
2,HW2,6.84272,5.77984,5.69518,1,HOH
3,MW,6.021258,6.023186,5.865343,1,HOH
4,OW,5.32179,5.32179,8.5662,2,HOH
...,...,...,...,...,...,...
539,MW,4.327824,12.992666,13.143288,135,HOH
540,OW,12.98252,4.31665,12.9933,136,HOH
541,HW1,12.44593,3.72957,13.52588,136,HOH
542,HW2,13.51643,3.73198,12.4554,136,HOH


In [45]:
# Now that I have a TIP4P hydrate, lets insert my THF into the large cages
large_cages = cages[cages['TYPE'].str.contains('Large')]
thf_center = thf[['X', 'Y', 'Z']].mean()
thf['RESIDUE_NUM'] = -1
thf['MOL'] = 'THF'

In [48]:
# This loop recenters THF and places it in all 8 large cages of the sII hydrate
thf_hydrate = tip4p_hydrate_total.copy()
for i in range(len(large_cages.index)):
    dxyz = thf_center - large_cages[['X','Y','Z']].iloc[i]
    thf_translated = thf.copy()
    thf_translated[['X','Y','Z']] = thf_translated[['X','Y','Z']] - dxyz
    thf_translated['RESIDUE_NUM'] = residue_num
    thf_hydrate = pd.concat([thf_hydrate, thf_translated], ignore_index=True)
    residue_num += 1

thf_hydrate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ELEMENT      648 non-null    object
 1   X            648 non-null    object
 2   Y            648 non-null    object
 3   Z            648 non-null    object
 4   RESIDUE_NUM  648 non-null    object
 5   MOL          648 non-null    object
dtypes: object(6)
memory usage: 30.5+ KB


In [49]:
# Write unit cell pdb
f = open("output/tip4p_thf_hydrate_unit.pdb", "w")

f.write("CRYST1   17.310   17.310   17.310  90.00  90.00  90.00 P 1           1\n")
for i in range(len(thf_hydrate.index)):
    #Collect PDB variables
    id = str(i)
    atom = thf_hydrate['ELEMENT'].iloc[i]
    resname = thf_hydrate['MOL'].iloc[i]
    resnum = thf_hydrate['RESIDUE_NUM'].iloc[i]
    x = thf_hydrate['X'].iloc[i]
    y = thf_hydrate['Y'].iloc[i]
    z = thf_hydrate['Z'].iloc[i]

    # Create PDB entry
    line = "{:6s}{:5d} {:^4s} {:3s}  {:4d}    {:8.3f}{:8.3f}{:8.3f}{:6.2f}{:6.2f}\n".format(
           "ATOM", i+1, atom, resname, resnum, x, y, z, 0., 0.)
    f.write(line)

f.close()

In [50]:
#Make the box 3 x 3
cell_size = 17.30
num_cells = 3
coord_offset = [[-cell_size, 0, cell_size],[0,0,cell_size],[cell_size, 0, cell_size],
                [-cell_size, 0, 0],[cell_size, 0, 0],
                [-cell_size, 0, -cell_size],[0, 0, -cell_size],[cell_size, 0, -cell_size]]

In [51]:
# Create 3x3 plane crystal
thf_hydrate_3_plane = thf_hydrate.copy()
for offset in coord_offset:
    residue_num = thf_hydrate_3_plane['RESIDUE_NUM'].iloc[len(thf_hydrate_3_plane.index) - 1]
    thf_hydrate_unit = thf_hydrate.copy()
    thf_hydrate_unit[['X','Y','Z']] = thf_hydrate_unit[['X','Y','Z']] + offset
    thf_hydrate_unit['RESIDUE_NUM'] = thf_hydrate_unit['RESIDUE_NUM'] + residue_num
    thf_hydrate_3_plane = pd.concat([thf_hydrate_3_plane, thf_hydrate_unit], axis=0, ignore_index=True)

In [52]:
# Stack 3x3x3 crystal
thf_hydrate_3_3 = thf_hydrate_3_plane.copy()
for i in range(1, num_cells):
    residue_num = thf_hydrate_3_3['RESIDUE_NUM'].iloc[len(thf_hydrate_3_3.index) - 1]
    thf_hydrate_plane_unit = thf_hydrate_3_plane.copy()
    thf_hydrate_plane_unit['Y'] = thf_hydrate_plane_unit['Y'] + (cell_size * i)
    thf_hydrate_plane_unit['RESIDUE_NUM'] = thf_hydrate_plane_unit['RESIDUE_NUM'] + residue_num
    thf_hydrate_3_3 = pd.concat([thf_hydrate_3_3, thf_hydrate_plane_unit], axis=0, ignore_index=True)


In [53]:
f = open("output/tip4p_thf_hydrate_3_3.pdb", "w")

f.write("CRYST1   52.000   52.000   52.000  90.00  90.00  90.00 P 1           1\n")
for i in range(len(thf_hydrate_3_3.index)):
    #Collect PDB variables
    id = str(i)
    chainid = 'A'
    atom = thf_hydrate_3_3['ELEMENT'].iloc[i]
    resname = thf_hydrate_3_3['MOL'].iloc[i]
    resnum = thf_hydrate_3_3['RESIDUE_NUM'].iloc[i]
    x = thf_hydrate_3_3['X'].iloc[i]
    y = thf_hydrate_3_3['Y'].iloc[i]
    z = thf_hydrate_3_3['Z'].iloc[i]

    # Create PDB entry
    line = "{:6s}{:5d} {:^4s} {:3s} {:1s}{:4d}    {:8.3f}{:8.3f}{:8.3f}{:6.2f}{:6.2f}\n".format(
           "ATOM", i+1, atom, resname, chainid, resnum, x, y, z, 0., 0.)
    f.write(line)

f.close()